<a href="https://colab.research.google.com/github/mamba124/just_for_fun/blob/master/Denoising_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow.keras.layers as layers
import tensorflow.keras.optimizers as optimizers
from tensorflow.keras import utils 
from tensorflow.keras.preprocessing import image 
import numpy as np 
import os 
import pandas as pd 
from tensorflow.keras.models import Model, load_model #из кераса подгружаем абстрактный класс базовой модели, метод загрузки предобученной модели

In [0]:
st = '/content/drive/My Drive/train/clean/47/47_122796_47-122796-0071 (1).npy'
stop = st[-7:]

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def get_chunks(signal, length, step):
  batch =[]
  syms_len = len(signal)
  index = 0
  
  while (index + length <= syms_len):
    batch.append(signal[index:index+length])
    index += step

  return batch

def prepare_chunks(audio,length, step):
  batch_list = get_chunks(audio, length, step)
  return np.array(batch_list)

In [0]:
directory = '/content/drive/My Drive/train/'
folders = os.listdir(directory)
clean_arrays = []
noisy_arrays = []
patch = os.listdir(directory+'clean/')

for folder in patch:
  for arr in os.listdir(directory+'clean/'+folder):
    if arr[-7:]!=stop:
      clean_arrays.append(np.load(directory+'clean/'+folder+'/'+arr))
      noisy_arrays.append(np.load(directory+'noisy/'+folder+'/'+arr))



In [0]:
clean_arrays_ = []
noisy_arrays_ = []

for i in range(len(clean_arrays[:-1])):
  for j in range(len(clean_arrays[i])):
    clean_arrays_.append(clean_arrays[i][j])
    noisy_arrays_.append(noisy_arrays[i][j])

In [0]:
length = 60 
step = 30

x_trainC = prepare_chunks(clean_arrays_, length, step)
x_trainN = prepare_chunks(noisy_arrays_, length, step)

y_trainC = np.ones((x_trainC.shape[0],1))
y_trainN = np.zeros((x_trainN.shape[0],1))

In [0]:
def shuffle(a, b):
    p = np.random.permutation(len(a)) 
    return a[p], b[p] 
x_train = np.concatenate([x_trainC, x_trainN], axis = 0)
y_train = np.concatenate([y_trainC, y_trainN], axis = 0)

#x_val = np.concatenate([x_valC, x_valN], axis = 0)
#y_val = np.concatenate([y_valC, y_valN], axis = 0)

x_train_, y_train_ = shuffle(x_train, y_train)
#x_val_, y_val_ = shuffle(x_val, y_val)

In [0]:
def hybrid(vector):

  inputs_vector = layers.Input(shape = (vector.shape[1],vector.shape[2]))

  ##Classifier##
  out_class = layers.BatchNormalization()(inputs_vector)
  out_class= layers.Conv1D(64, 3,strides = 2, padding = 'same')(out_class)
  out_class = layers.Flatten()(out_class)
  out_class = layers.Dense(200, activation='elu')(out_class)
  out_class = layers.Dense(100, activation='elu')(out_class)
  out_class = layers.Dense(56, activation = 'relu')(out_class)
  out_class = layers.Dense(1, activation = 'sigmoid')(out_class)
  
  ###Encoder###
  ##Block1##
  hidden = layers.Conv1D(4, 3, padding = 'same') (inputs_vector)
  hidden = layers.Activation('tanh')(hidden)
 # hidden = layers.BatchNormalization()(hidden)

  hidden = layers.Conv1D(4, 3, padding = 'same')(hidden)
  hidden = layers.Activation('relu')(hidden)
 # hidden = layers.BatchNormalization()(hidden)

  hidden = layers.MaxPooling1D(3)(hidden)

  ##Block2##
  hidden = layers.Conv1D(32, 3, padding = 'same') (hidden)
  #hidden = layers.BatchNormalization()(hidden)
  hidden = layers.Conv1D(32, 3, padding = 'same', strides = 2)(hidden)
  hidden = layers.Activation('relu')(hidden)
 # hidden = layers.BatchNormalization()(hidden)
  hidden = layers.MaxPooling1D(3)(hidden)
  
  ##Block3##
  hidden = layers.Conv1D(128, 3, padding = 'same') (hidden)
  #hidden = layers.BatchNormalization()(hidden)
  hidden = layers.MaxPooling1D(3)(hidden)  
  hidden = layers.Multiply()([1-out_class, hidden])
  
  ###Decoder###
  ##UPBlock1##
  hidden = layers.UpSampling1D(3)(hidden)
  hidden = layers.Conv1D(128, 3, padding = 'same', activation= 'relu')(hidden)
  hidden = layers.BatchNormalization()(hidden)

  ##UPBlock2##
  hidden = layers.UpSampling1D(4)(hidden)
  hidden = layers.Conv1D(32, 3, padding = 'same', activation= 'tanh')(hidden)
  hidden = layers.BatchNormalization()(hidden)
  hidden = layers.Conv1D(32, 3, padding = 'same', activation= 'tanh')(hidden)
  hidden = layers.BatchNormalization()(hidden)

  ##UPblock3##
  hidden = layers.UpSampling1D(5)(hidden)
  hidden = layers.Conv1D(4, 3, padding = 'same',  activation= 'tanh')(hidden)
  hidden = layers.BatchNormalization()(hidden)
  hidden = layers.Conv1D(4, 3, padding = 'same',  activation= 'tanh')(hidden)
  hidden = layers.BatchNormalization()(hidden)

 #out = layers.Conv1D(80, 3, padding = 'same',  activation= 'tanh')(hidden)
  hidden = layers.Flatten()(hidden)
  hidden = layers.Dense(156, activation='relu')(hidden)
  hidden = layers.Dense(256, activation='elu')(hidden)
  out = layers.Dense(60*80, activation = 'tanh')(hidden)
  out = layers.Reshape((60,80))(out)
  modelD = Model(inputs=inputs_vector, outputs = [out, out_class])
  #modelD.summary()

  return modelD


  

In [11]:
denoise = hybrid(x_train_)
denoise.compile(optimizer = optimizers.Adam(), loss = ['mse','binary_crossentropy'], metrics = ['mse','accuracy'])
denoise.fit(x_train_, [x_train_, y_train_], validation_split = 0.3, batch_size=500, epochs = 20)

Epoch 1/20
850/850 [==============================] - 79s 93ms/step - loss: 0.2822 - reshape_loss: 0.0699 - dense_3_loss: 0.2123 - reshape_mse: 0.0699 - reshape_accuracy: 0.1144 - dense_3_mse: 0.0625 - dense_3_accuracy: 0.9158 - val_loss: 0.2341 - val_reshape_loss: 0.0649 - val_dense_3_loss: 0.1693 - val_reshape_mse: 0.0649 - val_reshape_accuracy: 0.1208 - val_dense_3_mse: 0.0484 - val_dense_3_accuracy: 0.9364
Epoch 2/20
850/850 [==============================] - 109s 129ms/step - loss: 0.2105 - reshape_loss: 0.0551 - dense_3_loss: 0.1555 - reshape_mse: 0.0551 - reshape_accuracy: 0.1277 - dense_3_mse: 0.0445 - dense_3_accuracy: 0.9412 - val_loss: 0.2056 - val_reshape_loss: 0.0568 - val_dense_3_loss: 0.1488 - val_reshape_mse: 0.0568 - val_reshape_accuracy: 0.1261 - val_dense_3_mse: 0.0427 - val_dense_3_accuracy: 0.9432
Epoch 3/20
850/850 [==============================] - 110s 130ms/step - loss: 0.1842 - reshape_loss: 0.0507 - dense_3_loss: 0.1335 - reshape_mse: 0.0507 - reshape_accurac

In [0]:
directory_dataset = '/content/drive/My Drive/train/noisy/1006'#input('dataset:')
directory_result = './'+'results/'#directory_dataset+'/'
os.mkdir(directory_result + 'denoised/')

result = []
for curr_file in os.listdir(directory_dataset):

  arr = np.load(directory_dataset + '/' + curr_file)
  test = prepare_chunks(arr, length, step)
  denoised, classes = denoise.predict(test)
  curr_class = round(classes.sum()/classes.shape[0])
  result.append([curr_file, curr_class])

  if curr_class==0:
    signals = []
    for signal in np.expand_dims(denoised,0):
      glued = []
      for i in range(len(signal)):
        if i == 0: ind = 0
        else: ind = length - step
        for j in range(ind,length):
          glued.append(signal[i][j])
      signals.append(np.array(glued))
    np.save(directory_result + 'denoised/' + curr_file, np.array(signals))

result = np.array(result)
result = pd.DataFrame({'id':result[:,0], 'class':result[:,1]})
result.to_csv(directory_result + '/result.csv')

In [0]:
signals[0]


array([[ 0.53601253,  0.5166369 ,  0.5117932 , ..., -0.11528199,
        -0.11403016, -0.1863984 ],
       [ 0.51557004,  0.5034173 ,  0.48616666, ..., -0.10942012,
        -0.14916992, -0.19482179],
       [ 0.5485771 ,  0.51248306,  0.46705633, ..., -0.12809446,
        -0.14818464, -0.1906527 ],
       ...,
       [ 0.5150783 ,  0.4910716 ,  0.50734407, ..., -0.17790236,
        -0.1919307 , -0.2585448 ],
       [ 0.51168233,  0.5127557 ,  0.4845567 , ..., -0.16107182,
        -0.20467472, -0.23210245],
       [ 0.5169717 ,  0.5367661 ,  0.49197665, ..., -0.15959717,
        -0.20162193, -0.24758662]], dtype=float32)

In [0]:
test[0]

array([[ 0.8193  ,  0.894   ,  0.764   , ...,  0.06464 ,  0.0693  ,
        -0.03635 ],
       [ 0.9004  ,  0.8896  ,  0.791   , ..., -0.00354 ,  0.01195 ,
        -0.0827  ],
       [ 0.887   ,  0.8364  ,  0.803   , ..., -0.1115  , -0.1282  ,
        -0.2499  ],
       ...,
       [ 0.785   ,  0.8135  ,  0.81    , ...,  0.1342  ,  0.172   ,
         0.1184  ],
       [ 0.793   ,  0.86    ,  0.832   , ...,  0.0805  ,  0.09576 ,
         0.02042 ],
       [ 0.872   ,  0.8584  ,  0.8447  , ...,  0.02864 ,  0.001496,
        -0.09467 ]], dtype=float16)